In [58]:
from txdpy import get_letter,get_num
from datetime import datetime
from tqdm import tqdm
import re
import os
import math
import time
import numpy as np
import pandas as pd
import xlwings as xw
import PySimpleGUI as sg
import win32api,win32con

In [2]:
# 读取excel数据
def read_data(file_path,first_cell):
    app = xw.App(visible=False, add_book=False)
    wb = app.books.open(file_path) # 打开Excel文件
    sheet = wb.sheets[0] # 选择第0个表单
    data = sheet.range(first_cell).expand().value # 读取全表数据
    cols = data[0]
    df = pd.DataFrame(data[1:], columns=cols)
    return df

In [3]:
root_path = sg.popup_get_folder("请选择根目录文件夹")
print(root_path)

D:/DeskTop/Learn/6-Liz


In [4]:
listing_path = root_path + '/listing'

In [5]:
start_time = time.time()

In [6]:
listing_path

'D:/DeskTop/Learn/6-Liz/listing'

## 1.0 读取数据

In [7]:
# 读取新品在途进度表
old_time = time.time()
for file in os.listdir(root_path):
    if '$' not in file and '在途' in file and '进度表' in file and '结果' not in file:
        progress_transit_path = root_path + '/' + file
        print(progress_transit_path)
        df_progress_transit = pd.read_excel(progress_transit_path,sheet_name="各账号上架情况跟踪表-在途")
        print("size：{}".format(len(df_progress_transit)))
print("读取时间为%.2f秒钟"%(time.time()-old_time))

D:/DeskTop/Learn/6-Liz/【新品专项】在途新品模板进度表_2023-8-21.xlsx
size：10997
读取时间为3.52秒钟


In [8]:
# 读取MRP替换料总表
old_time = time.time()
for file in os.listdir(root_path):
    if '$' not in file and '替代料总表' in file and '_' in file:
        altersku_mrp_path = root_path + '/' + file
        print(altersku_mrp_path)
        df_altersku_mrp = pd.read_excel(altersku_mrp_path,header=0, sheet_name='MRP SKU')[['主料SKU',
                                                                                             '替换料1',
                                                                                             '替换料2',
                                                                                             '替换料3',
                                                                                             '替换料4',
                                                                                             '替换料5',
                                                                                             '替换料6',
                                                                                             '数量',
                                                                                             'SKU1',
                                                                                             'SKU2',
                                                                                             'SKU3',
                                                                                             'SKU4',
                                                                                             'SKU5']]
        
        df_altersku_mrp['SKU6'] = np.nan
        df_altersku_outmrp = pd.read_excel(altersku_mrp_path,header=0, sheet_name='MRP外SKU')[['主料SKU',
                                                                                                 '替换料1',
                                                                                                 '替换料2',
                                                                                                 '替换料3',
                                                                                                 '替换料4',
                                                                                                 '替换料5',
                                                                                                 '替换料6',
                                                                                                 '数量',
                                                                                                 'SKU1',
                                                                                                 'SKU2',
                                                                                                 'SKU3',
                                                                                                 'SKU4',
                                                                                                 'SKU5',
                                                                                                 'SKU6']]
        print("size：{}".format(len(df_altersku_mrp)))
print("读取时间为%.2f秒钟"%(time.time()-old_time))

D:/DeskTop/Learn/6-Liz/替代料总表_处理后.xlsx
size：56542
读取时间为10.52秒钟


In [9]:
# 读取listing
list_listing = []
old_time = time.time()
for file in os.listdir(listing_path):
    file_path = listing_path + '/' + file
    if "$" not in file:
        df = read_data(file_path,'A2')
        df = df[['ItemID', 'Seller', '主SKU', 'SKU',  '刊登方式(天)', '在线数量','刊登日期']]
        print("====== {}——读取完成 ======".format(file_path))
        list_listing.append(df)
# 合并
df_listing = pd.concat(list_listing)
print("读取时间为%.2f秒钟"%(time.time()-old_time))

====== D:/DeskTop/Learn/6-Liz/listing/2U-Listing20230911.xlsx——读取完成 ======
====== D:/DeskTop/Learn/6-Liz/listing/AP-Listing20230911.084613.xlsx——读取完成 ======
====== D:/DeskTop/Learn/6-Liz/listing/T-Listing20230911.085008.xlsx——读取完成 ======
====== D:/DeskTop/Learn/6-Liz/listing/大账号-Listing20230911.082519.xlsx——读取完成 ======
读取时间为50.62秒钟


In [10]:
# seller对应不同的账号名称，分别为：
# 2U: partswholesale2u
# Ap: a-premium-us
# US-T: total_autopart
# 大账号: autoparts_premium
# partswholesale2u,a-premium-us,total_autopart,autoparts_premium
df_listing.head()

,ItemID,Seller,主SKU,SKU,刊登方式(天),在线数量,刊登日期
0,2.853753e+11,partswholesale2u,P-USCMPS70300,P-USCMPS70300,一口价(GTC),25.0,2023-07-11 08:40:32
1,2.853753e+11,partswholesale2u,P-USABS20070,P-USABS20070,一口价(GTC),0.0,2023-07-11 08:30:19
2,2.853753e+11,partswholesale2u,P-USABS18460,P-USABS18460,一口价(GTC),25.0,2023-07-11 08:20:15
3,2.840760e+11,partswholesale2u,P-USGS45850TY,P-USGS45850TY,一口价(GTC),25.0,2020-11-11 09:18:42
4,2.853753e+11,partswholesale2u,P-USBHH04792,P-USBHH04792,一口价(GTC),25.0,2023-07-11 08:10:14


## 2.0 listiing处理

In [11]:
df_listing = df_listing.reset_index(drop=True)
df_listing = df_listing.drop_duplicates()
df_listing['ItemID'] = df_listing['ItemID'].apply(int)
'size：{}'.format(len(df_listing))

'size：271228'

In [12]:
list_sku=df_listing['主SKU'].to_list()

In [13]:
# 主SKU清洗
def clear_sku(list_skux):
    for i in range(0,len(list_skux)):
        a = re.sub('\（','(',list_skux[i])#转化(
        a1 = re.sub('\）',')',a)#转化）
        a2 = re.sub('\(.*?\)','',a1)#删除（）内信息
        a2 = re.sub('-测试','',a2)
        a2 = re.sub('，','',a2)
        a3 = re.sub(r'^[A-Z]{1,2}-{1}','',a2,count=1)#删除前缀
        a4 = re.sub('-NC','',a3)#删除-NC后缀-少数，USOS24423H-NC，USIC02680I-6-NC(质量投诉，需退回检测)
        a5 = re.sub('-N$','',a4)#删除-N后缀
        y=re.sub('-Scion$','',a5)
        z=re.sub(r'-Chevrolet$','',y)
        h = re.sub('-1\*2','*2',z)
        j =re.sub('USIC20181/USIC65500','USIC65500',h)
        k = re.sub('USBCB92767/USBC477563','USBC477563',j)
        l = re.sub('USHLS29918/USHLS60843','USHLS60843',k)
        m = re.sub('USGS436403','USGS43640B',l)
        m1=re.sub(r'USDS36847case停售10.19','USDS36847',m)
        n = re.sub('-US1$','',m1)
        o = re.sub('-Ford$','',n)
        p = re.sub('-MAP$','',o)
        q = re.sub('-Dodge$','',p)
        a6 = re.sub('-2021','',q)#删除后缀为-2021的SKU，USSGTE003-2021，USSGTE004-2021
        z=re.sub('case较高暂停出售','',a6)
        a7 = re.sub('-U$','',z)#删除后缀为-U的SKU，USGS47710-U，USIM15178-U先发测试
        a8 = re.sub('-4Runner$','',a7)#删除后缀为-4runner的SKU，USIFTO001Z-4Runner
        a9 = re.sub('-1A$','',a8)#删除后缀为‘-1A’的SKU,'-1AL'无误
        a10 = re.sub('-1B$','',a9)#删除后缀为‘-1B’的SKU
        a11 = re.sub('-1C$','',a10)#删除后缀为‘-1C’的SKU,'-1CA'无误
        a12 = re.sub('-1D$','',a11)#删除后缀为‘-1D’的SKU
        a13 = re.sub('-C1$','',a12)#删除后缀为‘-C1’的SKU
        a14 = re.sub('-M$','',a13)#删除后缀为‘-M’的SKU
        a15 = re.sub('-A$','',a14)#删除后缀为‘-A’的SKU
        a16 = re.sub('-B$','',a15)#删除后缀为‘-B’的SKU
        a17 = re.sub('-R2021$','-R',a16)
        a18 = re.sub('-L2021$','-L',a17)#删除后缀‘2021’
        a19= re.sub(' / ','/',a18)
        a20=re.sub('-$','',a19)
        a21=re.sub('-Rear$','',a20)
        a22=re.sub('-1.7kw$','',a21)
        a23=re.sub('-GMC$','',a22)
        a24=re.sub('-AMC$','',a23)
        a25=re.sub('-Chrysler$','',a24)
        a26=re.sub('-JPN$','',a25)
        a27=re.sub('-Other$','',a26)
        a28=re.sub('-US2$','',a27)
        a29=re.sub('-UR$','',a28)
        a29=re.sub('-UK$','',a29)
        a30=re.sub('-IT','',a29)
        a30=re.sub('-Ford$','',a30)
        a30=re.sub('-Left$','',a30)
        a30=re.sub('-Chevrolet$','',a30)
        a30=re.sub('-Dodge$','',a30)
        a30=re.sub('-N$','',a30)
        a30=re.sub('-D$','',a30)
        a32=re.sub('-DE$','',a30)
        a34=re.sub('-GMC1$','',a32)
        a35=re.sub('-GMC22$','',a34)
        a36=re.sub('-GMC2$','',a35)
        a37=re.sub('-Dodge Other$','',a36)
        a38=re.sub('-Dodge Ram$','',a37)
        a39=re.sub('-Oldsmobile$','',a38)
        a40=re.sub('-Reservoir$','',a39)
        a41=re.sub(r'-Chevy[0-9]{0,1}$','',a40)
        a42=re.sub(r'-UP$','',a41)
        a43=re.sub(r'-Return$','',a42)
        a44=re.sub(r'-MF$','',a43)
        a45=re.sub(r'-Front$','',a44)
        a46=re.sub(r'-Nissan$','',a45)
        a46=re.sub(r'-Toyota$','',a46)
        a46=re.sub(r'-VW$','',a46)
        a46=re.sub(r'-RH$','',a46)
        a46=re.sub(r'pin','',a46)
        a46=re.sub(r'-Case','',a46)
        a46=re.sub(r'-Saab','',a46)
        a46=re.sub(r'Note','',a46)
        a46=re.sub(r'\)','',a46)
        a46 = re.sub(r'[\u4e00-\u9fa5]', '', a46)
        a46 = re.sub(r'\-C','',a46)
        a46 = re.sub(r'\*','-',a46)
        a46=a46.rstrip('/')
        a46=a46.rstrip('+')
        a46=a46.strip()
        a46=a46.split('(')[0]
        list_skux[i]=a46
    return list_skux

In [14]:
list_sku = clear_sku(list_sku)
df_listing['处理后SKU'] = list_sku

In [15]:
# 处理异常UUS的SKU
index_u = df_listing[df_listing['处理后SKU'].str.startswith('UUS')].index.tolist()
for i in index_u:
    df_listing.loc[i,'处理后SKU'] = df_listing.loc[i,'处理后SKU'][1:]
    list_sku[i] = list_sku[i][1:]

In [16]:
# MRP替换料
# 将替换料关系变成每一行，对应相同的row，index
# 将组合料SKU，用“+”拼接起来
df_altersku_mrp_alt =  df_altersku_mrp
df_altersku_mrp_alt = df_altersku_mrp_alt.copy()
df_altersku_mrp_alt['替换料SKU'] = df_altersku_mrp_alt[df_altersku_mrp.columns[:7]].apply(lambda x: ','.join(x[x.notnull()]).split(','),axis=1)
df_altersku_mrp_alt['commbination_sku'] = df_altersku_mrp_alt[df_altersku_mrp.columns[8:-1]].apply(lambda x: '+'.join(x[x.notnull()]),axis=1)
df_altersku_mrp_alt_e = df_altersku_mrp_alt[['数量','替换料SKU', 'commbination_sku']]
df_altersku_mrp_alt_e = df_altersku_mrp_alt_e.explode('替换料SKU')
# df_altersku_mrp_alt_e

In [17]:
# 给替代料增加标识符row
index_list = list(df_altersku_mrp_alt_e.index)
index_row_list = [str(x+1) for x in index_list]
df_altersku_mrp_alt_e['index'] = index_list
df_altersku_mrp_alt_e['row'] = index_row_list
df_altersku_mrp_alt_e.columns = ['数量','SKU', 'commbination_sku','row','index']
df_altersku_mrp_alt_e = df_altersku_mrp_alt_e.drop_duplicates(keep='first')
df_altersku_mrp_alt_e = df_altersku_mrp_alt_e.reset_index(drop=True)
df_altersku_mrp_alt_e.head()

,数量,SKU,commbination_sku,row,index
0,1+1+1,USCA171876D-3,USCA51317D+USCA51318D+USCAB3376D,0,1
1,1+1+1,USCA171876D,USCA51317D+USCA51318D+USCAB3376D,0,1
2,1+1+1,USCA171876-3,USCA51317D+USCA51318D+USCAB3376D,0,1
3,1+1+1,USCA171876,USCA51317D+USCA51318D+USCAB3376D,0,1
4,1,USGS37022T,,1,2


In [18]:
df_listing_dazhanghao  = df_listing[df_listing['Seller']=='autoparts_premium'].reset_index(drop=True)
df_listing_dazhanghao = df_listing_dazhanghao[['Seller','处理后SKU','刊登日期']]
df_listing_dazhanghao.columns = ['Seller','SKU', '刊登日期']
df_listing_dazhanghao = df_listing_dazhanghao.copy()
df_listing_dazhanghao['刊登日期'] = pd.to_datetime(df_listing_dazhanghao['刊登日期'], errors='coerce')
# df_listing_dazhanghao.head()

In [19]:
# 将MRP外的SKU，处理之后的拼接起来，得到outsku
df_altersku_outmrp_tmp = df_altersku_outmrp
df_altersku_outmrp_tmp['outsku'] = df_altersku_outmrp_tmp[df_altersku_outmrp_tmp.columns[8:]].apply(lambda x: '+'.join(x[x.notnull()]), axis=1)
df_altersku_outmrp_tmp.head()

,主料SKU,替换料1,替换料2,替换料3,替换料4,替换料5,替换料6,数量,SKU1,SKU2,SKU3,SKU4,SKU5,SKU6,outsku
0,USHLS41285+USHLS41286,NaN,NaN,NaN,NaN,NaN,NaN,1+1,USHLS41285,USHLS41286,NaN,NaN,NaN,NaN,USHLS41285+USHLS41286
1,USDLA31004+USDLA31005+USDLA31006+USDLA31007,USDLA31004+DLA31005+DLA31006+DLA31007,NaN,NaN,NaN,NaN,NaN,1+1+1+1,USDLA31004,USDLA31005,USDLA31006,USDLA31007,NaN,NaN,USDLA31004+USDLA31005+USDLA31006+USDLA31007
2,USDLA31008+USDLA31009,NaN,NaN,NaN,NaN,NaN,NaN,1+1,USDLA31008,USDLA31009,NaN,NaN,NaN,NaN,USDLA31008+USDLA31009
3,USDLA76319+USDLA76544,NaN,NaN,NaN,NaN,NaN,NaN,1+1,USDLA76319,USDLA76544,NaN,NaN,NaN,NaN,USDLA76319+USDLA76544
4,USDLA20273+USDLA20303,NaN,NaN,NaN,NaN,NaN,NaN,1+1,USDLA20273,USDLA20303,NaN,NaN,NaN,NaN,USDLA20273+USDLA20303


In [20]:
# 将大账号listing中MRP没有的SKU进行匹配，然后更新为处理之后的
df_account = df_listing_dazhanghao
df_account = pd.merge(left=df_account, 
                      right=df_altersku_outmrp_tmp, 
                      how='left',
                      left_on='SKU', 
                      right_on='主料SKU')
df_account['SKU'] = [df_account.loc[i,'outsku'] if pd.notna(df_account.loc[i,'outsku']) else df_account.loc[i,'SKU'] for i in range(df_account.shape[0])]
df_account = df_account[['Seller', 'SKU', '刊登日期']]
df_account.head()

,Seller,SKU,刊登日期
0,autoparts_premium,USBRPS258485FR+USBCB73130LR+USBCB72928BLR,2023-03-11 08:53:32
1,autoparts_premium,USCTP00003,2023-03-11 08:29:30
2,autoparts_premium,USABS308AAJ+USABS316AAJ+USABS317ABJ+USABS509AA,2015-11-11 04:30:15
3,autoparts_premium,USCAK556LRD+USCAK091LRD,2020-03-11 01:16:03
4,autoparts_premium,USFP3621S0Z,2016-08-10 22:26:19


In [21]:
# 将每一个组合SKU根据“+”号分开，然后每个SKU去寻找是否有对应的替代料
def new_sku(values):
    values = values.split('+')
    
    for value in values:
        element_to_find = value
        row_index, col_index = (df_altersku_mrp_alt_e == element_to_find).values.nonzero()
        if len(row_index)>0:
            row_index = row_index[0]
            commbination_sku = df_altersku_mrp_alt_e.loc[row_index,'commbination_sku']
            
            if isinstance(commbination_sku, pd.DataFrame) or isinstance(commbination_sku, pd.Series):
                if commbination_sku.iloc[0]!='':
                    values.remove(value)
                    values.append(commbination_sku.iloc[0])
                    
            elif isinstance(commbination_sku, str):
                if commbination_sku!='':
                    values.remove(value)
                    values.append(commbination_sku)
                    
    return set(values)

In [22]:
a = 'USBRPS258485FR+USBCB73130LR+USBCB72928BLR'
b = new_sku(a)
print(b)

{'USBCB73130LR', 'USBCB84728B-C+USBCB84729B-C', 'USBRS55072+USBPD78400+USBRS55055+USBPD78500'}


In [50]:
# 筛选新品在途表中大账号刊登日期为空的，并且匹配替代料表，匹配新品在途表每个SKU是否有对应的替代料关系
df_progress_transit_notnull = df_progress_transit[pd.isna(df_progress_transit['大账号'])][['SKU', '大账号', 'ap', '2u', 'total']].reset_index(drop=True)
df_progress_transit_notnull = pd.merge(left=df_progress_transit_notnull, right=df_altersku_mrp_alt_e, how='left', left_on='SKU', right_on='SKU')
# df_progress_transit_notnull

In [60]:
# 1 - 排除没有row值的，即为没有对应的替代料关系
# 2 - 再筛选对应的数量关系，排除掉数量为1以及空的，因为这些不存在组合料，只有替代料关系
df_progress_transit_notnull_mul = df_progress_transit_notnull[df_progress_transit_notnull['row'].notna()].fillna('')
df_progress_transit_notnull_mul = df_progress_transit_notnull_mul[(df_progress_transit_notnull_mul['数量']!=1)&(df_progress_transit_notnull_mul['数量']!='')]
# df_progress_transit_notnull_mul

In [70]:
tqdm.pandas()
df_progress_transit_notnull_mul['new_commbination_transit'] = df_progress_transit_notnull_mul['commbination_sku'].progress_apply(new_sku)
tqdm.close()
# df_progress_transit_notnull_mul

In [74]:
df_account_alter = pd.merge(left=df_account, right=df_altersku_mrp_alt_e, how='left', left_on='SKU', right_on='SKU')
df_account_alter = df_account_alter[(pd.notna(df_account_alter['row']))&(df_account_alter['数量']!=1)]
# df_account_alter

In [79]:
df_account_alter['new_commbination_account'] = df_account_alter['commbination_sku'].progress_apply(new_sku)
df_account_alter

In [96]:
df_account_alter.iloc[2257]